## Importing Dependencies

In [1]:
import time    #for calculating time
import math    #for using floor in creating minibatches
import pickle  #for saving model


#core packages
import numpy as np
import matplotlib.pyplot as plt

#custom module
from  dataset import load_dataset, train_dev_split, prep_dataset
from dataset import visualize_data_distribution, visualize_dataset, label_description

# from finalModelUtils import relu, relu_grad, softmax

np.random.seed(1)
%matplotlib inline

## Dataset Preparation

### Loading Dataset

In [2]:
dataset_size_in_per = 100

train_x_orig, train_y_orig, test_x_orig, test_y_orig = load_dataset(dataset = "mnist", size_in_per = dataset_size_in_per)

print("Sample Size : %d%%\n"%(dataset_size_in_per))
print("Data\t\t\t","Datatype\t\t","Dataset Size")
print("=================================================================")
print("Training Set Images:\t" + str(type(train_x_orig))+"\t",str(train_x_orig.shape))
print("Training Set Labels:\t" + str(type(train_y_orig))+"\t",str(train_y_orig.shape))
print("Test Set Images:\t" + str(type(test_x_orig))+"\t",str(test_x_orig.shape))
print("Test Set Labels:\t" + str(type(test_y_orig))+"\t",str(test_y_orig.shape))
print("=================================================================")



Sample Size : 100%

Data			 Datatype		 Dataset Size
Training Set Images:	<class 'numpy.ndarray'>	 (60000, 28, 28)
Training Set Labels:	<class 'numpy.ndarray'>	 (60000, 1)
Test Set Images:	<class 'numpy.ndarray'>	 (10000, 28, 28)
Test Set Labels:	<class 'numpy.ndarray'>	 (10000, 1)


### Train-Dev set Split

In [3]:
train_x_split, train_y_split, dev_x_split, dev_y_split = train_dev_split(train_x_orig, train_y_orig)

print("Data\t\t\t\t","Datatype\t\t","Shape")
print("========================================================================")
print("Training Set Images:\t\t" + str(type(train_x_split))+"\t",str(train_x_split.shape))
print("Training Set Labels:\t\t" + str(type(train_y_split))+"\t",str(train_y_split.shape))
print("Development Set Images:\t\t" + str(type(dev_x_split))+"\t",str(dev_x_split.shape))
print("Development Set Labels:\t\t" + str(type(dev_y_split))+"\t",str(dev_y_split.shape))
print("========================================================================")


Data				 Datatype		 Shape
Training Set Images:		<class 'numpy.ndarray'>	 (50000, 28, 28)
Training Set Labels:		<class 'numpy.ndarray'>	 (50000, 1)
Development Set Images:		<class 'numpy.ndarray'>	 (10000, 28, 28)
Development Set Labels:		<class 'numpy.ndarray'>	 (10000, 1)


### Preparing Dataset

In [6]:
train_x_norm, train_y_encoded = prep_dataset(train_x_split, train_y_split, num_class = 10)
dev_x_norm, dev_y_encoded= prep_dataset(dev_x_split, dev_y_split, num_class = 10)
test_x_norm, test_y_encoded = prep_dataset(test_x_orig, test_y_orig, num_class = 10)

print("Data\t\t\t","Before Processing\t","After Processing")
print("=================================================================")
print("Training Set Images:\t" + str(train_x_split.shape)+"\t\t"+ str(train_x_norm.shape))
print("Training Set Labels:\t" + str(train_y_split.shape)+"\t\t"+ str(train_y_encoded.shape))
print("Dev Set Images:\t\t" + str(dev_x_split.shape)+"\t\t"+ str(dev_x_norm.shape))
print("Dev Set Labels:\t\t" + str(dev_y_split.shape)+"\t\t"+ str(dev_y_encoded.shape))
print("Test Set Images:\t" + str(test_x_orig.shape)+"\t\t"+ str(test_x_norm.shape))
print("Test Set Labels:\t" + str(test_y_orig.shape)+"\t\t"+ str(test_y_encoded.shape))
print("=================================================================")

Data			 Before Processing	 After Processing
Training Set Images:	(50000, 28, 28)		(784, 50000)
Training Set Labels:	(50000, 1)		(10, 50000)
Dev Set Images:		(10000, 28, 28)		(784, 10000)
Dev Set Labels:		(10000, 1)		(10, 10000)
Test Set Images:	(10000, 28, 28)		(784, 10000)
Test Set Labels:	(10000, 1)		(10, 10000)


## Model Utils

### ReLU Activation

In [7]:
#relu function
def relu(Z):
    """Compute the ReLU activation of Z.
        
        Arguments:
            Z (numpy.ndarray): Input Sum to a hidden unit, Z = W * X + b.
        
        Returns:
            tuple: Following values
            - **A** (numpy.ndarray): Activation obtained by applying ReLU function to the sum. Size same as that of Z.
            - **cache** (numpy.ndarray): Value stored for use during backward propagation.
        
        Example:
            >>> np.random.seed(1)
            >>> Z = np.random.randn(1,6)
            >>> **A,cache = relu(Z)**
            >>> print(A)
            
            Output: 
                [[1.62434536 0.     0.      0.      0.86540763 0.    ]]
    """

    A = np.maximum(0.0,Z)
    
    cache = Z #storing Z for later use during back propagation
    assert(A.shape == Z.shape)
    return A, cache


In [8]:
#relu gradient function
def relu_grad(dA, cache):
    """Compute the backward ReLU activation of dA.
        
        Arguments:
            dA (numpy.ndarray): Gradient of activation of the previous layer.
            cache (numpy.ndarray): Value of Z stored during forward prop.
            
        Returns:
            numpy.ndarray: - **dZ**: array of gradient/derivative of the dA, Same size of dA.
            
        Example:
            >>> np.random.seed(1)
            >>> dA = np.random.randn(1,6)
            >>> cache = np.random.randn(1,6)
            >>> **dZ = relu_grad(dA,cache)**
            >>> print(dZ)
            
            Output:
                [[ 1.62434536  0.         -0.52817175  0.          0.86540763  0.        ]]
    """
    
    Z = cache
    dZ = np.array(dA, copy=True) 
    
    dZ[Z <= 0] = 0 #implementing integrated form of (gradiant of ReLU function * gradient of the loss function)
    
    assert(dZ.shape == Z.shape)
    return dZ


### Softmax Activation

In [9]:
def softmax(Z):
    """Compute the softmax activtion of Z.
        
        Argument:
            Z (numpy.ndarray): Input Sum to a hidden unit, Z = W * X + b.
        
        Returns:
            tuple: Following Values
            - **A** (numpy.ndarray): Activation obtained by applying softmax function to the sum. Size same as that of Z.
            - **cache** (numpy.ndarray): Value stored for use during backward propagation.
            
        Example:
            >>> np.random.seed(2)
            >>> Z= np.random.rand(7,1)
            >>> **A,cache = softmax(Z)**
            >>> print(A)
            
            Output:
                [[0.15477477]
                 [0.10270926]
                 [0.17340649]
                 [0.15467071]
                 [0.15237489]
                 [0.13925557]
                 [0.12280831]]
    """
    shift = Z - np.max(Z, axis = 0) #Avoiding underflow or overflow errors due to floating point instability in softmax
    t = np.exp(shift)
    A = np.divide(t,np.sum(t,axis = 0))
    
    cache = Z
    assert(A.shape == Z.shape)
    return A, cache



Testing 
```
z = np.asarray([[1,2,3,6],[2,4,5,6],[1,2,3,6]],dtype='float32')
z = z.T
print(z)
a,c = softmax(z)
print(a)
print(np.sum(a, axis = 0),end = '\n\n\n\n')

print("-"*40)
#ref link https://exceptionshub.com/how-to-implement-the-softmax-function-in-python.html

z = np.asarray([[1,2,3,6],[2,4,5,6],[1,2,3,6]],dtype='float32')
print(z)
s = np.max(z, axis=1).reshape((-1, 1))
print(s)
e_x = np.exp(z - s)
print(e_x)
div = np.sum(e_x, axis=1).reshape((-1, 1))
print(div)
a =  e_x / div

print(a)
print(np.sum(a, axis = 1))
```

```
import tensorflow as tf
import numpy as np
print(tf.__version__)
z = np.asarray([[1,2,3,6],[2,4,5,6],[1,2,3,6]],dtype='float32')
print(z, z.shape)
print(z.T, z.T.shape)

batch = tf.constant(z)
print(batch)
y = tf.nn.softmax(batch)

print(y)


# Find the largest value
print(tf.reduce_max(batch))
# Find the index of the largest value
print(tf.argmax(batch))
# Compute the softmax
print(tf.nn.softmax(batch))

```

```
from sklearn.utils.extmath import softmax
import numpy as np
z = np.asarray([[1,2,3,6],[2,4,5,6],[1,2,3,6]],dtype='float32').reshape(3,4)

softmax(z)
```

### Creating Minibatches

In [10]:
def rand_mini_batches(X, Y, minibatch_size = 64, seed=1):
    """Returns the minibatches of X and corresponding Y of the given size.
    
        Arguments:
            X (numpy.ndarray): Inputs Array.
            Y (numpy.ndarray): Output Labels.
            minibatch_size (int): Size of each minibatch.
            seed (int): Seed value for randomness.
        
        Returns:
            list: - **minibatches**: List of minibatches where each minibatch contains a minibatch of X and a minibatch of its corresponding Y.
            
        Examples:
            >>> X = np.random.randn(20,20)
            >>> Y = np.random.rand(1,20)
            >>> **minibatches = rand_mini_batches(X,Y,minibatch_size = 4, seed = 2)**
            >>> print(minibatches[0][0].shape)
            >>> print(minibatches[0][1].shape)
            
            Outputs:
                (20, 4)
                (1, 4)
    """
   
    classes = Y.shape[0]
    np.random.seed(seed) # varying the seed value so that the minibatchs become random in each epoch
    m = X.shape[1]       # number of training examples
    minibatches = []
        
    #Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((classes,m))

    #Partition (shuffled_X, shuffled_Y) except for the last batch
    num_complete_minibatches = math.floor(m/minibatch_size) # number of mini batches of size minibatch_size 
    for k in range(0, num_complete_minibatches):
        minibatch_X = shuffled_X[:, k * minibatch_size : (k+1)*minibatch_size]
        minibatch_Y = shuffled_Y[:, k * minibatch_size : (k+1)*minibatch_size]
        minibatch = (minibatch_X, minibatch_Y)
        minibatches.append(minibatch)
    
    # Last batch (last minibatch <= minibatch_size)
    if m % minibatch_size != 0:
        minibatch_X = shuffled_X[:, num_complete_minibatches * minibatch_size : m]
        minibatch_Y = shuffled_Y[:, num_complete_minibatches * minibatch_size : m]
        minibatch = (minibatch_X, minibatch_Y)
        minibatches.append(minibatch)
    
    return minibatches

### Time Conversion

In [11]:
#time Conversion
def convert_time(millisec):
    """Converts time in miliseconf to higher values.
    
        Arguments:
            milisec (int): Time in mili-seconds.
        
        Returns: 
            tuple: Following values
            - **hours** (int):  Time in hours.
            - **mins** (int): Time in minutes.
            - **secs** (int):  Time in seconds.
            - **milisec** (int): Time in mili-seconds.
            
        Example:
            >>> **hr,mins,sec,milisec = convert_time(millisec = 12450)**
            >>> print("%dhr %dmins %ds %dms"%(hr,mins,sec,milisec))
            
            Outputs:
                0hr 0mins 12s 450ms
    """
    #converting millisecons to hours, minutes, seconds and millisecond
    #the large numbers like 3.6e+6 comes from the relation between the time units
    hours = millisec // 3.6e+6
    mins = (millisec % 3.6e+6) // 60000
    secs = ((millisec % 3.6e+6) % 60000) // 1000
    millisec = ((millisec % 3.6e+6) % 60000) % 1000
    
    return (hours, mins, secs, millisec)


### Computing and Visualizing Evaluation Matrices

In [12]:
#creating Confusion Matrix
def confusion_matrix(y_orig,prediction):
    """Returns a confusion matrix for a given output labels and prediction.
    
        Arguments:
            y_orig (numpy.ndarray): Original Output Labels of shape(m,1); m = # of examples.
            prediction (numpy.ndarray): Predicted Labels of the dataset of shape (1,m).
        
        Returns:
            numpy.ndarray:- **cm**: 2D confusion matrix.
            
        Example:
            >>> y = np.array([[1,2,3,3,0,2,5,4,2,2]]).reshape(10,1)
            >>> pred = np.array([[2,2,1,3,0,2,5,4,2,2]]).reshape(1,10)
            >>> prediction = {"First Prediction":pred}
            >>> **cm_train = confusion_matrix(y,prediction)**
            >>> print(cm_train)
            
            Output:
                [[1 0 0 0 0 0]
                 [0 0 1 0 0 0]
                 [0 0 4 0 0 0]
                 [0 1 0 1 0 0]
                 [0 0 0 0 1 0]
                 [0 0 0 0 0 1]]
    """
    first_predict = prediction["First Prediction"]
    
    y_predicted = first_predict[0].T
    
    m = y_orig.shape[0]
    classes = len(np.unique(y_orig)) # or simply take classes = 10 for mnist or fashion-mnist
    
    cm = np.zeros((classes,classes)) #creating the matrix frame for the confusion matrix
    
    # generating the values in the confusion metrix
    for i in range(m):
        cm[y_orig[i],y_predicted[i]] += 1
   
    return cm.astype(int)
    

In [13]:
## Plotting Confusion Matrix

def plot_confusion_matrix(cm, dataset_type, dataset = "mnist"):
    """Plots the Heatmap of the confusion matrix.
    
        Arguments:
            cm (numpy.ndarray): 2D confusion matrix.
            dataset_type (str): Type of dataset. May be training or dev or test.
            dataset (str): Dataset used to train the model. Default to 'mnist'
            
        Example:
            >>> plot_confusion_matrix(cm, dataset_type)
    """
    # plotting the metrix
    fig, ax = plt.subplots(figsize=(10,10))
    im = plt.imshow(cm,cmap="GnBu") #RdYlGn, PiYG, Accent,Blues,viridis, YlGnBu
    
    # plotting the color bar of the plot size
    fig.colorbar(im,ax=ax,fraction=0.045)
    
    if(len(dataset_type) != 0):
        visual_title = "Confusion Matrix for %s Set "%dataset_type.capitalize()
    else:
        raise ValueError("Dataset set must be training or dev or test set")
        
        
    #getting the label description
    label_desc = label_description(dataset)
    desc = [label_desc[i] for i in range(0,10)]
    
    # annotating the plot
    ax.set_title(visual_title,fontsize=24,pad = 20)    
    ax.set_xticks(np.arange(10))
    ax.set_yticks(np.arange(10))    
    ax.set_xlabel("Predicted", fontsize = 20)
    ax.set_ylabel("Expexted", fontsize = 20)
    
    ax.set_xticklabels(desc)
    ax.set_yticklabels(desc)
    
    #setting horizontal axes labeling to top.
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')

    # creating the threshold for color change in the visualization
    thres = cm.max()//2
    
    for i in range(10):
        for j in range(10):
            #calculating the percentage of the total image denoted by the cell
            per = cm[i,j]/cm.sum() * 100
            #putting up the text inside the plot cells
            ax.text(j, i, "%d\n%.2f%%"%(cm[i, j], per),
                           ha="center", va="center", color="w" if cm[i,j] > thres else "black")


    fig.tight_layout()
    plt.show()


In [14]:
## Calculating precision, Recall and F1-Score
def precision(label, cm):
    """Returns the precision for the prediction of an individual label.
    
        Arguments:
            label (int): unique labels (each class).
            cm (numpy.ndarray): 2D confusion matrix.
        
        Returns:
            numpy.float64: - **prec**: Precision for the given label.
            
        Example:
            if cm = [[1 0 0 0 0 0]
                     [0 0 1 0 0 0]
                     [0 0 4 0 0 0]
                     [0 1 0 1 0 0]
                     [0 0 0 0 1 0]
                     [0 0 0 0 0 1]]
            >>> label = 2
            >>> prec = precision(label, cm)
            >>> print(prec)
            
            Output:
                0.8
    """
    col = cm[:, label] #selecting the True Positive and false positive values
    prec = cm[label, label] / col.sum() #perc = TP / (TP + FP)
    return prec
    
def recall(label, cm):
    """Returns the recall for the prediction of an individual label.
    
        Arguments:
            label (int): unique labels (each class).
            cm (numpy.ndarray): 2D confusion matrix.
        
        Returns:
            numpy.float64: - **rec**: Recall for the given label.
            
        Example:
            if cm = [[1 0 0 0 0 0]
                     [0 0 1 0 0 0]
                     [0 0 4 0 0 0]
                     [0 1 0 1 0 0]
                     [0 0 0 0 1 0]
                     [0 0 0 0 0 1]]
            >>> label = 2
            >>> rec = recall(label, cm)
            >>> print(rec)
            
            Output:
                1.0
    """
    row = cm[label, :] #selecting the True Positive and false Negative values
    rec = cm[label, label] / row.sum() # rec = TP / (TP + FN)
    return rec

def f1_score(prec,rec):
    """Returns the f1-score for the prediction of an individual label.
    
        Arguments:
            prec (numpy.float64): precision for a label.
            rec (numpy.float64): recall for a label.
        
        Returns:
            numpy.float64: - **f1**: f1-score for the precision and recall.
            
        Example:
            >>> prec = 0.8
            >>> rec = 1.0
            >>> f1 = f1_score(prec,rec)
            >>> print(f1)
            
            Output:
                0.888888888888889
    """
    f1 = (2 * prec * rec) / (prec + rec)
    return f1


In [15]:
##Calculating macro precision, recall and f1-score
def macro_precision_average(precs):
    """Returns the macro average of the precisions for the prediction of all the label.
    
        Arguments:
            precs (list): lists of precisions of type numpy.float64.
        
        Returns:
            numpy.float64: - **prec_mac_avg**: macro average of the precisions.
            
        Example:
            >>> avg_precision = macro_precision_average(prec)
    """
    count = len(precs)    
    prec_mac_avg = np.sum(precs) / count
    return prec_mac_avg

def macro_recall_average(recs):
    """Returns the macro average of the recall for the prediction of all the label.
    
        Arguments:
            recs (list): lists of recalls of type numpy.float64.
        
        Returns:
            numpy.float64: - **rec_mac_avg**: macro average of the recalls.
            
        Example:
            >>> avg_recall = macro_recall_average(rec)
    """
    count = len(recs)
    rec_mac_avg = np.sum(recs) / count
    return rec_mac_avg

def macro_f1_score(f1s):
    """Returns the macro average of the f1-score for the prediction of all the label.
    
        Arguments:
            f1s (list): lists of f1-scores of type numpy.float64.
        
        Returns:
            numpy.float64: - **f1_mac_avg**: macro average of the f1-scores.
            
        Example:
            >>> avg_f1 = macro_f1_score(f1)
    """
    count = len(f1s)
    f1_mac_avg = np.sum(f1s) / count
    return f1_mac_avg


In [16]:
##calculating the accuracy
def accuracy(cm):
    """Returns the accuracy of the prediction.
    
        Arguments:
            cm (numpy.ndarray): 2D confusion matrix.

        Returns:
            numpy.float64: - **acc**: Accuracy of the prediction.
            
        Example:
            if cm = [[1 0 0 0 0 0]
                     [0 0 1 0 0 0]
                     [0 0 4 0 0 0]
                     [0 1 0 1 0 0]
                     [0 0 0 0 1 0]
                     [0 0 0 0 0 1]]
            >>> acc = accuracy(cm)
            
            Outputs:
                0.8
    """
    diagonal_sum = cm.trace() # getting the total truely classified images
    sum_of_all_elements = cm.sum() # getting the total number of images
    acc = diagonal_sum / sum_of_all_elements 
    return acc


In [17]:
## Calculating model metrices
def model_metrics(cm):
    """Returns the metrices and macro metrices for the evaluation of the model.
        Metrices includes:
            Precision, Recall and F1-score
        Macro Metrices includes:
            Macro Precision Average, Macro Recall Average and Macro F1-score Average
    
        Arguments:
            cm (numpy.ndarray): 2D confusion matrix.

        Returns: tuple: Following Values
            - **metrices** (dict): Model Metrices including list of Precision, Recall and F1-score.
            - **macro_metrices** (dict): Model Macro metrices including Macro Precision Average, Macro Recall Average and Macro F1-score Average.
            - **acc** (numpy.float64): Accuracy of the prediction.
            
        Example:
            >>> metrics, macro_metrics, acc = model_metrics(cm)
    """
    precs = []
    recs = []
    f1s = []
    metrics = {}
    macro_metrics = {}
    # calculating precision, recall and f1-score for all the classes or labels
    for label in range(10):
        precs.append(precision(label, cm))
        recs.append(recall(label, cm))
        f1s.append(f1_score(precs[label], recs[label]))
    
    #calculating the macro average metrices
    avg_precision = macro_precision_average(precs) #calculating the macro metrices
    avg_recall = macro_recall_average(recs)
    avg_f1 = macro_f1_score(f1s)
    acc = accuracy(cm)
    
    metrics = {"Precision":precs,
               "Recall":recs,
               "F1-Score":f1s}
    macro_metrics = {"Precision":avg_precision,
                     "Recall":avg_recall,
                     "F1-Score":avg_f1}
    
    return metrics, macro_metrics, acc


In [18]:
##displaying the model summary
def metric_summary(metrics, macro_metrics, accuracy):
    """Displays the metric summary after evaluation.
    
        Arguments:
            metrices (dict): Model Metrices including list of Precision, Recall and F1-score.
            macro_metrices (dict): Model Macro metrices including Macro Precision Average, Macro Recall Average and Macro F1-score Average.
            accuracy (numpy.float64): Accuracy of the prediction.
            
        Example:
            >>> metric_summary(metrics, macro_metrics, acc)
    """
    print("+===============+===============+===============+===============+")
    print("| Label \t| Precision \t| Recall \t| F1 Score \t|")
    print("+===============+===============+===============+===============+")
    prec = metrics["Precision"]
    rec = metrics["Recall"]
    f1 = metrics["F1-Score"]
    
    for label in range(len(prec)):
        print("| %d \t\t|  %.5f \t|  %.5f \t|  %.5f \t|"%(label, prec[label], rec[label], f1[label]))

    print("+===============+===============+===============+===============+") 
    
    avg_precision = macro_metrics["Precision"]
    avg_recall = macro_metrics["Recall"]
    avg_f1 = macro_metrics["F1-Score"]
    acc = accuracy
    
    print("| Macro Avg \t|  %.5f \t|  %.5f \t|  %.5f \t|"%( avg_precision, avg_recall, avg_f1))
    print("+===============+===============+===============+===============+") 
    
    print("\n Accuracy \t\t  %.5f"%(acc))



### Visualizing Predictions and acc-loss plots

In [19]:
# Visualizing the Training Result

def visualize_training_results(train_accs, val_accs, train_loss, val_loss):
    """Visualize the traininig accuracy and loss, validation accuracy and loss over the training time.
    
        Arguments:
            train_accs (list): training accuracies obtained in all the minibatches in all epochs
            val_accs (list): validation accuracies obtained in all the minibatches in all epochs
            train_loss (list): training losses obtained in all the minibatches in all epochs
            val_loss (list): validation losses obtained in all the minibatches in all epochs
            
        Example:
            >>> visualize_training_results(train_accs, val_accs, train_losses, val_losses)    
    """
    
    #creating subplots
    fig, axes = plt.subplots(nrows=2, ncols = 1,figsize=(10,15))
    fig.subplots_adjust(wspace=.2, hspace = .5)
    
    #plotting the loss
    axes[0].plot(np.squeeze(train_loss), label = 'Training Loss', color = 'blue')
    axes[0].plot(np.squeeze(val_loss), label = 'Validation Loss', color = 'red')
    axes[0].legend(loc='upper right') #setting up legend location to upper right corner of the plot
    axes[0].set_title("Training and Validation Loss " , fontsize = 16, pad = 10)
    axes[0].set_xlabel("No. of Epochs", fontsize = 12)
    axes[0].set_ylabel("Loss", fontsize = 12)
    axes[0].set_ylim(bottom = 0)  
    axes[0].grid(color='grey', alpha = 0.5)
    
    #plotting the accuracy
    axes[1].plot(np.squeeze(train_accs), label = 'Training Accuracy', color = 'blue')
    axes[1].plot(np.squeeze(val_accs), label = 'Validation Accuracy', color = 'red')
    axes[1].legend(loc='lower right') #setting up legend location to lower right corner of the plot
    axes[1].set_title("Accuracy " , fontsize = 16, pad = 10)
    axes[1].set_xlabel("No. of Epochs", fontsize = 12)
    axes[1].set_ylabel("Accuracy", fontsize = 12)
    axes[1].set_ylim(top = 1)  
    axes[1].grid(color='grey', alpha = 0.5)

    plt.show()


In [20]:
#Visualizing Prediction
def visualize_prediction(x_orig, y_orig, prediction, dataset_type, dataset = "mnist"):
    """Displays 10 random images along with their true and predicted labels. 
        Both initial prediction and second guess are displayed.
    
        Arguments:
            x_orig (numpy.ndarray): original input data.
            y_orig (numpy.ndarray): original output labels.
            prediction (numpy.ndarray): predictions obtained after training the model.
            dataset_type (str): Type of dataset. May be training, dev  or test.
            dataset (str): Dataset used to train the model. Default to 'mnist'
            
        Example:
            >>> visualize_prediction(x_orig, y_orig.T, prediction, dataset_type = "training")
    """
    if(len(dataset_type) != 0):
        visual_title = "Sample %s Data Set "%dataset_type.capitalize()
    else:
        raise ValueError("Dataset set must be training or dev or test set")
    
    #getting the random index of 8 images to plot
    index = np.random.randint(0,1000,8)
    
    #getting the label description
    label_desc = label_description(dataset)
    
    #plotting the images along with the predictions
    fig, axes = plt.subplots(nrows=2, ncols=4,figsize=(16,8))
    fig.subplots_adjust(hspace=0.5)
    fig.suptitle(visual_title)

    first_lbl, first_prob = prediction["First Prediction"]
    sec_lbl, sec_prob = prediction["Second Prediction"]

    for ax,i in zip(axes.flatten(),index):
        ax.imshow(x_orig[i].squeeze(),interpolation='nearest', cmap='Greys')
        ax.set(title = "True Label: %d | %s"%(y_orig[0,i], label_desc[y_orig[0,i]] ))
        ax.set(xlabel= "Prediction: %d | With Prob: %.4f \n2nd Guess: %d | With Prob: %.4f"%(first_lbl[0,i], first_prob[0,i], sec_lbl[0,i], sec_prob[0,i]))


In [21]:
# Visualizing Mislabeled Data
def visualize_mislabelled_images(x_orig,y_orig,prediction,dataset_type, dataset = "mnist"):
    """Displays 10 wrongly predicted images along with their true and predicted labels. 
        Both initial prediction and second guess are displayed.
    
        Arguments:
            x_orig (numpy.ndarray): original input data.
            y_orig (numpy.ndarray): original output labels.
            prediction (numpy.ndarray): predictions obtained after training the model.
            dataset_type (str): Type of dataset. May be training, dev  or test.
            dataset (str): Dataset used to train the model. Default to 'mnist'
            
        Example:
            >>> visualize_mislabelled_images(x_orig, y_orig.T, prediction, dataset_type = "training")
    """
    
    first_lbl, first_prob = prediction["First Prediction"]
    sec_lbl, sec_prob = prediction["Second Prediction"]
    
    true_prediction = np.equal(first_lbl,y_orig)
    mislabelled_indices = np.asarray(np.where(true_prediction == False))
    print("Total Mislabelled Images: "+str(len(mislabelled_indices[0])))
    
    if(len(dataset_type) != 0):
        visual_title = "Sample Mislabelled %s Set Images "%dataset_type.capitalize()
    else:
        raise ValueError("Dataset set must be training or dev or test set")
    
    #getting the label description
    label_desc = label_description(dataset)     
    
    #plotting the mislabelled images along with the predictions
    fig, axes = plt.subplots(nrows=2, ncols=5,figsize=(16,8))
    fig.subplots_adjust(hspace=1)
    fig.suptitle(visual_title)

    for ax,i in zip(axes.flatten(),mislabelled_indices[1]):
        ax.imshow(x_orig[i].squeeze(),interpolation='nearest', cmap = "Greys")
        ax.set(title = "True Label: %d | %s"%(y_orig[0,i], label_desc[y_orig[0,i]] ))
        ax.set(xlabel= "Prediction: %d | With Prob: %.4f \n2nd Guess: %d | With Prob: %.4f"%(first_lbl[0,i], first_prob[0,i], sec_lbl[0,i], sec_prob[0,i]))


### Saving and Loading Model

In [22]:
#saving Model
def save_model(file_name, model):
    """Saves the parameters of the trained model into a pickle file.
    
        Arguments:
            file_name (str): name of the file to be saved.
            model (dict): trained model to be saved in the file. Consists of Parameters and activations 
            
        Example:
            >>> save_model(file_name = path + fname, model)
    """
    with open(file_name ,'wb') as output_file:
        pickle.dump(model,output_file)


In [23]:
#Loading Model
def load_model(file_name):
    """Load the saved model from a pickle file
    
        Arguments:
            file_name (str): name of the file to be loaded from
        
        Returns:
            model (dict): trained model to be saved in the file. Consists of Parameters and activations 
            
        Example:
            >>> model = load_model(file_name = path + fname)
    """
    try: 
        with open(file_name ,'rb') as input_file:
            model = pickle.load(input_file)
        
        return model
    
    except(OSError, IOError) as e:
        print(e)


## Model Implementation

### Initializing the Model

#### Initializing Layers

In [24]:
def init_layers(X,Y,hidden_layers):
    """
        
        Arguments:
            
            
        Returns:
            
            
        Example:
            Here, shape of x = (784,m)
                  shape of y = (10,m)
            >>> layers_dim = init_layers(x, y, hidden_layers = [32,16])
            >>> print(layers_dim)
            
            Outputs:
                [784, 32, 16, 10]
    """
    input_nodes = X.shape[0]
    output_nodes = Y.shape[0]
    
    layers_dim = [input_nodes]
    
    for i in hidden_layers:
        layers_dim.append(i)
    
    layers_dim.append(output_nodes)
    
    return layers_dim

#### Initializing Parameters

In [25]:
def init_parameters(layers_dim, initialization = "random"):
    """
        
        Arguments:
            
            
        Returns:
            
            
        Example:
            Here, layers_dim = [784, 32, 16, 10]
            >>> parameters = init_parameters(layers_dim, initialization = "random")
            >>> print("Layer\tWeight\t\tBias")
            >>> print("================================")
            >>> for l in range(1,len(layers_dim)):
            ...     print(str(l) +"\t" + str(parameters['W'+str(l)].shape) +"\t"+ str(parameters['b'+str(l)].shape))

            
            Outputs:
                Layer    Weight         Bias
                ================================
                1        (32, 784)      (32, 1)
                2        (16, 32)       (16, 1)
                3        (10, 16)       (10, 1)
    """
    L = len(layers_dim)
    params = {}
        
    for l in range(1,L):
        #initializing Weights
        if initialization == "he":
            # he-initialization
            params['W' + str(l)] = np.random.randn(layers_dim[l],layers_dim[l-1]) * np.sqrt(np.divide(2,layers_dim[l-1])) 
        elif initialization == "random":
            # random initialization scaled by 0.01
            params['W' + str(l)] = np.random.randn(layers_dim[l],layers_dim[l-1]) * 0.01 
        else:
             raise ValueError("Initialization must be 'random' or 'he'")
        
        #initializing biases
        params['b' + str(l)] = np.zeros((layers_dim[l],1))
     
        assert(params['W' + str(l)].shape == (layers_dim[l],layers_dim[l-1])), "Dimention of W mismatched in init_params function"
        assert(params['b' + str(l)].shape == (layers_dim[l],1)), "Dimention of b mismatched in init_params function"
   
    return params

#### Initializing Hyper Parameters

In [26]:
def init_hyperParams(alpha, num_epoch, minibatch_size, lambd = 0, keep_probs = []):
    """
        
        Arguments:
            
            
        Returns:
            
            
        Example:
            >>> hyperParams = init_hyperParams(alpha = 0.0001, num_epoch = 10, minibatch_size = 1024,lambd = 0.7,keep_probs = [0.8,0.8])
    """
    hyperParams = {'learning_rate':alpha,
                   'num_epoch':num_epoch,
                   'mini_batch_size':minibatch_size,
                   'lambda':lambd,
                   'keep_probs':keep_probs,
                   'beta1':0.9,
                   'beta2':0.999,
                   'epsilon':1e-8
                  }
    
    return hyperParams

### Forward Propagation

#### Forward Sum for individual Layer

In [27]:
def forward_sum(A_prev,W,b):
    """
    
    
        Example:
            >>> np.random.seed(1)
            >>> A = np.random.randn(3,2)
            >>> W = np.random.randn(1,3)
            >>> b = np.random.randn(1,1)
            >>> Z, c = forward_sum(A,W,b)
            >>> print("Z = "+ str(Z))
            
            Output:
                Z = [[ 3.26295337 -1.23429987]]
        
    """
    m = A_prev.shape[1]
    
    Z = np.dot(W,A_prev) + b
    
    cache = (A_prev,W,b)
    
    assert (Z.shape == (W.shape[0], m)), "Dimention of Z mismatched in forward_prop function"
    
    return Z, cache

#### Forward Activation for individual Layer

In [28]:
def forward_activation(A_prev,W,b,activation):
    """
    
    
        Example:
            >>> np.random.seed(1)
            >>> A_prev = np.random.randn(3,2)
            >>> W = np.random.randn(1,3)
            >>> b = np.random.randn(1,1)

            >>> A,c = forward_activation(A_prev,W,b,activation = 'relu')
            >>> print("A with Relu = " + str(A))

            >>> A,c = forward_activation(A_prev,W,b,activation = 'softmax')
            >>> print("A with Softmax = " + str(A))
            
            Output:
                A with Relu = [[3.26295337 0.        ]]
                A with Softmax = [[1. 1.]]
    """
    
    if activation == 'relu':
        Z, sum_cache = forward_sum(A_prev,W,b)
        A, activation_cache = relu(Z)
        
    elif activation == 'softmax':
        Z, sum_cache = forward_sum(A_prev,W,b)
        A, activation_cache = softmax(Z)
    
    elif activation == "tanh":
#         Z, sum_cache = forward_sum(A_prev,W,b)
#         A, activation_cache = tanh(Z)
        pass
    
    cache = (sum_cache,activation_cache)
    
    assert(A.shape == Z.shape), "Dimention of A mismatched in forward_activation function"
    
    return A, cache

#### Dropout for individual Layer

In [29]:
def forward_dropout(A,keep_probs):
     #implementing dropout
    D = np.random.rand(A.shape[0],A.shape[1])
    D = (D < keep_probs).astype(int)
    A = np.multiply(A,D)
    A = np.divide(A,keep_probs)
    
    dropout_mask = D
    
    assert (dropout_mask.shape == A.shape), "Dimention of dropout_mask mismatched in forward_dropout function"
    return A,dropout_mask

#### Forward Prop for L Layers

In [30]:
def forward_prop(X, parameters, keep_probs = [], regularizer = None):
    """
    
        Example:
            >>> np.random.seed(1)
            >>> X = np.random.randn(3,2)
            >>> W1 = np.random.randn(3,3)
            >>> b1 = np.random.randn(3,1)
            >>> W2 = np.random.randn(2,3)
            >>> b2 = np.random.randn(2,1)
            >>> parameters = {"W1": W1,
                              "b1": b1,
                              "W2": W2,
                              "b2": b2}
            >>> AL, caches, _ = forward_prop(X, parameters)
            >>> print("AL without dropout = " + str(AL))

            >>> AL, caches, _ = forward_prop(X, parameters,keep_probs = [0.9], regularizer = "dropout")
            >>> print("\nAL with dropout = " + str(AL))

            >>> print("\nLength of caches list = " + str(len(caches)))
            
            Output:
                AL without dropout = [[0.25442549 0.64096177]
                 [0.74557451 0.35903823]]

                AL with dropout = [[0.20251119 0.61487938]
                 [0.79748881 0.38512062]]

                Length of caches list = 2
    
    """
    caches = []
    A = X
    L = len(parameters) // 2
    num_class = parameters["W"+str(L)].shape[0]
    
    dropout_masks = []

    # len(keep_probs) == L-1: no dropouts in the Output layer, no dropout at all for prediction
    if regularizer == "dropout":
        assert(len(keep_probs) == L-1 ) 
    
    for l in range(1, L):
        A_prev = A 
        A, cache = forward_activation(A_prev,parameters['W' + str(l)],parameters['b' + str(l)], activation='relu')
        caches.append(cache)
        if regularizer == "dropout":
            A , dropout_mask = forward_dropout(A,keep_probs[l-1])
            dropout_masks.append(dropout_mask)
        else:
            pass

    AL, cache = forward_activation(A, parameters['W' + str(L)], parameters['b' + str(L)], activation='softmax')
    caches.append(cache)
    
    assert(AL.shape == (num_class,X.shape[1])), "Dimention of AL mismatched in forward_prop function"
    
    return AL,caches,dropout_masks
    

### Compute Cost

In [31]:
def softmax_cross_entropy_cost(AL, Y, caches, lambd = 0, regularizer = None, from_logits = False ):
    """
    
    
        Example:
            >>> AL = np.array([[4.21200131e-01, 1.55876995e-04],
                           [6.91917292e-02, 1.18118501e-05],
                           [5.09608140e-01, 9.99832311e-01]])
            >>> cost = softmax_cross_entropy_cost(AL, Y, caches)
            >>> print("Cost without l2 = " + str(cost))

            >>> cost = softmax_cross_entropy_cost(AL, Y, caches, lambd = 0.7, regularizer = 'l2')
            >>> print("Cost with l2 = " + str(cost))
            
            Output:
                Cost without l2 = 0.6742809046007259
                Cost with l2 = 8.875542970361
    """
    
    L = len(caches)
    m = Y.shape[1]
    
     #cost computation from logit
    #ref link : https://www.d2l.ai/chapter_linear-networks/softmax-regression-concise.html
    if from_logits == True:
        z = caches[-1][-1] #retriving the logit(activation cache) of the last layer from the caches
        s
        z = z - np.max(z,axis = 0) #calculating negative z for avoiding numerical overflow in exp computation
        logit_log =  z - np.log(np.sum(np.exp(z),axis = 0)) #calculating the log of the softmax to feed into cost
        cost = -(1./m) * np.sum(np.sum(np.multiply(Y,logit_log), axis = 0,keepdims=True))
        
    else:
        cost = -(1./m) * np.sum(np.sum(np.multiply(Y,np.log(AL + 1e-8)), axis = 0,keepdims=True))# add very small number 1e-8 to avoid log(0)
ss
    if regularizer == "l2":
        norm = 0
        for l in range(L):
            current_cache = caches[l]
            sum_cache, _ = current_cache
            _,W,_ = sum_cache
            norm += np.sum(np.square(W))

        L2_cost = (lambd/(2*m)) * norm 
        cost = cost + L2_cost
    else:
        pass
    
    cost = np.squeeze(cost)      # Making sure your cost's shape is not returned as ndarray
    
    assert(cost.shape == ()),"Dimention of cost mismatched in softmax_cross_entropy_cost function"
    
    return cost

### Backward Propagation

#### Calculating Gradients for individual Layer

In [32]:
def backward_grad(dZ, cache, lambd, regularizer):
    """
    
        Example:
            >>> np.random.seed(1)
            >>> dZ = np.random.randn(3,4)
            >>> A = np.random.randn(5,4)
            >>> W = np.random.randn(3,5)
            >>> b = np.random.randn(3,1)
            >>> cache = (A, W, b)

            >>> dA_prev, dW, db = backward_grad(dZ, cache, lambd=0, regularizer=None)
            >>> print("Without L2 Regularization")
            >>> print ("dA_prev = "+ str(dA_prev))
            >>> print ("dW = " + str(dW))
            >>> print ("db = " + str(db))

            >>> l2_dA_prev, l2_dW, l2_db = backward_grad(dZ, cache, lambd = 0.9, regularizer = 'l2')
            >>> print("\nWith L2 Regularization")
            >>> print ("dA_prev = "+ str(l2_dA_prev))
            >>> print ("dW = " + str(l2_dW))
            >>> print ("db = " + str(l2_db))
            
            Output:
                Without L2 Regularization
                dA_prev = [[-1.15171336  0.06718465 -0.3204696   2.09812712]
                           [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
                           [-0.4319552  -1.30987417  1.72354705  0.05070578]
                           [-0.38981415  0.60811244 -1.25938424  1.47191593]
                           [-2.52214926  2.67882552 -0.67947465  1.48119548]]
                dW = [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
                      [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
                      [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
                db = [[-0.14713786]
                      [-0.11313155]
                      [-0.13209101]]

                With L2 Regularization
                dA_prev = [[-1.15171336  0.06718465 -0.3204696   2.09812712]
                           [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
                           [-0.4319552  -1.30987417  1.72354705  0.05070578]
                           [-0.38981415  0.60811244 -1.25938424  1.47191593]
                           [-2.52214926  2.67882552 -0.67947465  1.48119548]]
                dW = [[-0.0814752  -0.28784277 -1.02688866  0.73478408 -0.24353767]
                      [ 0.90783172  0.74875962 -0.43216662  0.6696189  -0.78903459]
                      [ 0.81102242  0.13703735 -0.07696496  0.4081879  -0.05995309]]
                db = [[-0.14713786]
                      [-0.11313155]
                      [-0.13209101]]
    """
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    if regularizer == "l2":
        dW = (1/m) * np.dot(dZ,A_prev.T) + np.multiply(np.divide(lambd,m),W )
    else:
        dW = (1/m) * np.dot(dZ,A_prev.T)

    db = (1/m) * np.sum(dZ, axis = 1, keepdims=True )
    dA_prev = np.dot(W.T, dZ)

    
    assert (dW.shape == W.shape), "Dimention of dW mismatched in backward_grad function"
    assert (db.shape == b.shape), "Dimention of db mismatched in backward_grad function"
    assert (dA_prev.shape == A_prev.shape), "Dimention of dA_prev mismatched in backward_grad function"
    
    
    return dA_prev, dW, db

#### Calculating Backward Activation for individual layer

In [33]:
def backward_activation(dA, cache, lambd ,regularizer, activation):
    """
    
    
    Example:
        >>> np.random.seed(2)
        >>> dA = np.random.randn(1,2)
        >>> A = np.random.randn(3,2)
        >>> W = np.random.randn(1,3)
        >>> b = np.random.randn(1,1)
        >>> Z = np.random.randn(1,2)
        >>> sum_cache = (A, W, b)
        >>> activation_cache = Z
        >>> cache = (sum_cache, activation_cache)

        >>> dA_prev, dW, db = backward_activation(dA, cache, lambd = 0 ,regularizer = None, activation = "relu")
        >>> print("With Relu")
        >>> print ("dA_prev = "+ str(dA_prev))
        >>> print ("dW = " + str(dW))
        >>> print ("db = " + str(db))

        >>> dA_prev, dW, db = backward_activation(dA, cache, lambd = 0 ,regularizer = None, activation = "softmax")
        >>> print("\nWith Softmax")
        >>> print ("dA_prev = "+ str(dA_prev))
        >>> print ("dW = " + str(dW))
        >>> print ("db = " + str(db))
        
        Output: 
            With Relu
            dA_prev = [[ 0.44090989 -0.        ]
                       [ 0.37883606 -0.        ]
                       [-0.2298228   0.        ]]
            dW = [[ 0.44513824  0.37371418 -0.10478989]]
            db = [[-0.20837892]]

            With Softmax
            dA_prev = [[ 0.44090989  0.05952761]
                       [ 0.37883606  0.05114697]
                       [-0.2298228  -0.03102857]]
            dW = [[ 0.39899183  0.3973954  -0.06975568]]
            db = [[-0.23651234]]
    """
    
    
    
    sum_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_grad(dA,activation_cache)
        dA_prev, dW, db = backward_grad(dZ, sum_cache, lambd, regularizer = regularizer)
        
    elif activation == "softmax":
        dZ = dA
        dA_prev, dW, db = backward_grad(dZ, sum_cache, lambd, regularizer = regularizer)
    
    elif activation == "tanh":
        pass
#         dZ = tanh_grad(dA,activation_cache)
#         dA_prev, dW, db = backward_grad(dZ, sum_cache, lambd, regularizer = regularizer)
    
    return dA_prev, dW, db
    
    

#### Complete Backward Propagation for L layers

In [34]:
def backward_dropout(dA_prev_temp, D, keep_prob):
    dA_prev = np.multiply(dA_prev_temp,D)
    dA_prev = np.divide(dA_prev,keep_prob)
    return dA_prev

In [35]:
def backward_prop(AL, Y, caches, dropout_masks = [], keep_probs = [], lambd = 0, regularizer = None):
    """
    
        Example:
            >>> np.random.seed(3)
            >>> AL = np.random.randn(1, 2)
            >>> Y = np.array([[1, 0]])

            >>> A1 = np.random.randn(4,2)
            >>> W1 = np.random.randn(3,4)
            >>> b1 = np.random.randn(3,1)
            >>> Z1 = np.random.randn(3,2)
            >>> cache_activation_1 = ((A1, W1, b1), Z1)

            >>> A2 = np.random.randn(3,2)
            >>> W2 = np.random.randn(1,3)
            >>> b2 = np.random.randn(1,1)
            >>> Z2 = np.random.randn(1,2)
            >>> cache_activation_2 = ((A2, W2, b2), Z2)

            >>> caches = (cache_activation_1, cache_activation_2)

            >>> grads = backward_prop(AL, Y, caches)
            >>> for key,value in grads.items():
            ...     print(str(key)+" : "+str(value))
            
            Output:
                dA1 : [[-0.80745758 -0.44693186]
                       [ 0.88640102  0.49062745]
                       [-0.10403132 -0.05758186]]
                dW2 : [[ 0.50767257 -0.42243102 -1.15550109]]
                db2 : [[0.61256916]]
                dA0 : [[ 0.          0.53064147]
                       [ 0.         -0.3319644 ]
                       [ 0.         -0.32565192]
                       [ 0.         -0.75222096]]
                dW1 : [[0.41642713 0.07927654 0.14011329 0.10664197]
                       [0.         0.         0.         0.        ]
                       [0.05365169 0.01021384 0.01805193 0.01373955]]
                db1 : [[-0.22346593]
                       [ 0.        ]
                       [-0.02879093]]
    """
    
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    dA = np.subtract(AL,Y)
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = backward_activation(dA, current_cache,lambd = lambd, regularizer = regularizer, activation = 'softmax')
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        
        if regularizer == "dropout":
            #implementing dropout
            D = dropout_masks[l]
            dA_prev_temp = backward_dropout(grads["dA" + str(l + 1)], D, keep_probs[l])
            dA_prev, dW_temp, db_temp = backward_activation(dA_prev_temp, current_cache, lambd = lambd, regularizer = regularizer, activation = 'relu')
        else:
            dA_prev, dW_temp, db_temp = backward_activation(grads["dA" + str(l + 1)], current_cache, lambd = lambd, regularizer = regularizer, activation = 'relu')
            
        
        grads["dA" + str(l)] = dA_prev
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

### Update Parameters

In [36]:
#initialize adam

def initialize_adam(parameters) :
   
    L = len(parameters) // 2 
    v = {}
    s = {}
    
    for l in range(L):
        v["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
        v["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
        s["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
        s["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
    
    return v, s

In [37]:
def update_parameters(parameters, grads, learning_rate, optimizer = "bgd", beta1 = 0, beta2 = 0,  epsilon = 0, v = {}, s = {}, t = 0):
    """
    
    
        Example:
            >>> np.random.seed(2)
            >>> W1 = np.random.randn(3,4)
            >>> b1 = np.random.randn(3,1)
            >>> W2 = np.random.randn(1,3)
            >>> b2 = np.random.randn(1,1)
            >>> parameters = {"W1": W1,
                          "b1": b1,
                          "W2": W2,
                          "b2": b2}
            >>> np.random.seed(3)
            >>> dW1 = np.random.randn(3,4)
            >>> db1 = np.random.randn(3,1)
            >>> dW2 = np.random.randn(1,3)
            >>> db2 = np.random.randn(1,1)
            >>> grads = {"dW1": dW1,
                     "db1": db1,
                     "dW2": dW2,
                     "db2": db2}

            >>> parameters,_,_ = update_parameters(parameters, grads, 0.1)

            >>> print ("W1 = "+ str(parameters["W1"]))
            >>> print ("b1 = "+ str(parameters["b1"]))
            >>> print ("W2 = "+ str(parameters["W2"]))
            >>> print ("b2 = "+ str(parameters["b2"]))
            
            Output:
                W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
                      [-1.76569676 -0.80627147  0.51115557 -1.18258802]
                      [-1.0535704  -0.86128581  0.68284052  2.20374577]]
                b1 = [[-0.04659241]
                      [-1.28888275]
                      [ 0.53405496]]
                W2 = [[-0.55569196  0.0354055   1.32964895]]
                b2 = [[-0.84610769]]
    """
    L = len(parameters) // 2           
    v_corrected = {}                         
    s_corrected = {}                       
    
    for l in range(L):
        if optimizer == 'adam':
            # Moving average of the gradients.
            v["dW" + str(l+1)] = np.add(beta1 * v["dW" + str(l+1)], (1 - beta1) * grads["dW" + str(l+1)])
            v["db" + str(l+1)] = np.add(beta1 * v["db" + str(l+1)], (1 - beta1) * grads["db" + str(l+1)])

            # Compute bias-corrected first moment estimate.
            v_corrected["dW" + str(l+1)] = np.divide(v["dW" + str(l+1)], (1 - np.power(beta1,t)))
            v_corrected["db" + str(l+1)] = np.divide(v["db" + str(l+1)], (1 - np.power(beta1,t)))

            # Moving average of the squared gradients. 
            s["dW" + str(l+1)] = np.add(beta2 * s["dW" + str(l+1)], (1 - beta2) * np.square(grads["dW" + str(l+1)]))
            s["db" + str(l+1)] = np.add(beta2 * s["db" + str(l+1)], (1 - beta2) * np.square(grads["db" + str(l+1)]))

            # Compute bias-corrected second raw moment estimate. 
            s_corrected["dW" + str(l+1)] = np.divide(s["dW" + str(l+1)], (1 - np.power(beta2,t)))
            s_corrected["db" + str(l+1)] = np.divide(s["db" + str(l+1)], (1 - np.power(beta2,t)))

            # Update parameters. 
            parameters["W" + str(l+1)] = np.subtract(parameters["W" + str(l+1)],  learning_rate * np.divide(v_corrected["dW" + str(l+1)], np.sqrt(s_corrected["dW" + str(l+1)]) + epsilon))
            parameters["b" + str(l+1)] = np.subtract(parameters["b" + str(l+1)],  learning_rate * np.divide(v_corrected["db" + str(l+1)], np.sqrt(s_corrected["db" + str(l+1)]) + epsilon))
        else:
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate * grads["dW" + str(l+1)])
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate * grads["db" + str(l+1)])
            
    return parameters, v, s

### Evaluate Model

In [38]:
# Evaluating the model using acc and loss
def evaluate(X, Y, parameters):
    """
    
        Example:
            >>> np.random.seed(1)
            >>> X = np.random.randn(3,2)
            >>> Y = np.array([[1, 0, 0],[0,1,1]]).reshape(3,2)
            >>> W1 = np.random.randn(5,3)
            >>> b1 = np.random.randn(5,1)
            >>> W2 = np.random.randn(3,5)
            >>> b2 = np.random.randn(3,1)
            >>> parameters = {"W1": W1,
            ...               "b1": b1,
            ...               "W2": W2,
            ...               "b2": b2}
            >>> acc, loss = evaluate(X, Y, parameters)
            >>> print("acc = %f | cost = %f"%(acc,loss))
            
            Output:
                acc = 0.500000 | cost = 0.769464
    """
    
    m = Y.shape[1]
    
    # predicting output using fordward propogation 
    probas, caches, _ = forward_prop(X, parameters)
    #computing loss
    loss = softmax_cross_entropy_cost(probas, Y, caches) 
    
    #deriving the predictrueted labels
    true_labels = np.argmax(Y,axis=0).reshape(1,m)
    #deriving the predicted labels
    predicted_labels = np.argmax(probas,axis=0).reshape(1,m)
    
    #identifing correctly predicted labels
    correct_prediction = np.equal(predicted_labels,true_labels)
    
    #computing accuracy
    num_correct_prediction = np.sum(correct_prediction)
    accuracy = (num_correct_prediction/m)
    
    return accuracy, loss